In [4]:
%%writefile omp_min_max_avg.cu

#include <iostream>
#include <omp.h>
#include <ctime>
#include <cstdlib>

using namespace std;

void min(int *arr, int n)
{
   double min_val = 10000;
   int i;
   cout << endl;
#pragma omp parallel for reduction(min : min_val)
   for (i = 0; i < n; i++)
   {
      cout << "\nthread id = " << omp_get_thread_num() << " and i = " << i;
      if (arr[i] < min_val)
      {
         min_val = arr[i];
      }
   }
   cout << "\n\nmin_val = " << min_val << endl;
}

void max(int *arr, int n)
{
   double max_val = 0.0;
   int i;

#pragma omp parallel for reduction(max : max_val)
   for (i = 0; i < n; i++)
   {
      cout << "\nthread id = " << omp_get_thread_num() << " and i = " << i;
      if (arr[i] > max_val)
      {
         max_val = arr[i];
      }
   }
   cout << "\n\nmax_val = " << max_val << endl;
}

void avg(int *arr, int n)
{
   int i;
   float avg = 0, sum = 0;
#pragma omp parallel reduction(+:sum)
   {
#pragma omp for
      for (i = 0; i < n; i++)
      {
         sum = sum + arr[i];
         cout << "\nthread id = " << omp_get_thread_num() << " and i = " << i;
      }
   }
   cout << "\n\nSum = " << sum << endl;
   avg = sum / n;
   cout << "\nAverage = " << avg << endl;
}

int main()
{
   omp_set_num_threads(4);
   int n, i;

   cout << "Enter the number of elements in the array: ";
   cin >> n;
   int *arr = new int[n];

   srand(time(0));
   for (int i = 0; i < n; ++i)
   {
      arr[i] = rand() % 100;
   }

   cout << "\nArray elements are: ";
   for (i = 0; i < n; i++)
   {
      cout << arr[i] << ",";
   }

   min(arr, n);
   max(arr, n);
   avg(arr, n);

   delete[] arr;
   return 0;
}



Writing omp_min_max_avg.cu


In [6]:
!nvcc omp_min_max_avg.cu -o min_max_avg


/usr/bin/ld: /tmp/tmpxft_00000a09_00000000-11_omp_min_max_avg.o: in function `min(int*, int)':
tmpxft_00000a09_00000000-6_omp_min_max_avg.cudafe1.cpp:(.text+0x7d): undefined reference to `omp_get_thread_num'
/usr/bin/ld: /tmp/tmpxft_00000a09_00000000-11_omp_min_max_avg.o: in function `max(int*, int)':
tmpxft_00000a09_00000000-6_omp_min_max_avg.cudafe1.cpp:(.text+0x198): undefined reference to `omp_get_thread_num'
/usr/bin/ld: /tmp/tmpxft_00000a09_00000000-11_omp_min_max_avg.o: in function `avg(int*, int)':
tmpxft_00000a09_00000000-6_omp_min_max_avg.cudafe1.cpp:(.text+0x2e4): undefined reference to `omp_get_thread_num'
/usr/bin/ld: /tmp/tmpxft_00000a09_00000000-11_omp_min_max_avg.o: in function `main':
tmpxft_00000a09_00000000-6_omp_min_max_avg.cudafe1.cpp:(.text+0x3e0): undefined reference to `omp_set_num_threads'
collect2: error: ld returned 1 exit status


In [8]:
!./omp_min_max_avg


Enter the number of elements in the array: 5

Array elements are: 0,33,31,22,69,

thread id = 3 and i = 4
thread id = 0 and i = 0
thread id = 0 and i = 1
thread id = 2 and i = 3
thread id = 1 and i = 2

min_val = 0

thread id = 0 and i = 0
thread id = 0 and i = 1
thread id = 2 and i = 3
thread id = 1 and i = 2
thread id = 3 and i = 4

max_val = 69

thread id = 0 and i = 0
thread id = 0 and i = 1
thread id = 2 and i = 3
thread id = 1 and i = 2
thread id = 3 and i = 4

Sum = 155

Average = 31
